# 3-DaskDataframe

This notebook demonstrates how to read and process a tabular datafile with the [Dask](https://docs.dask.org/en/stable/dataframe.html) Dataframe library. A Dask DataFrame is a large parallel DataFrame composed of many smaller Pandas DataFrames. Dask can handle datasets that are large than the available memory (out-of-core) and process them in parallel on multiple cores.

Author: Peter W. Rose (pwrose@ucsd.edu)

In [1]:
import os
from dask.distributed import Client
import dask.dataframe as dd
import time

In [2]:
#from jupyter_client import KernelManager
#from jupyter_client import KernelManager
#KernelManager.shutdown_wait_time = 60

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [3]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")

### Setup Benchmark

The ```n_cores``` and ```file_format``` parameter are used for benchmarking ([see](7-ParallelEfficiency.ipynb)). 
This Cell [3] has been [parameterized](https://papermill.readthedocs.io/en/latest/usage-parameterize.html) as input parameters for [papermill](https://papermill.readthedocs.io/en/latest/index.html).

In [4]:
n_cores = 0 # default -> uses all available cores
file_format = "csv"

In [5]:
start = time.time()

### Read Data

In [6]:
if n_cores > 0:
    # use n_cores for benchmarking
    client = Client(n_workers=n_cores)
else:
    # use all available cores
    client = Client()

In [7]:
# read only specified columns
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id", "chromosome"]

if file_format == "csv":
    filename = os.path.join(DATA_DIR, "gene_info.tsv")
    # = dd.read_csv(filename, usecols=column_names, dtype=str, sep="\t", blocksize="0.25 GB")
    genes = dd.read_csv(filename, usecols=column_names, dtype=str, sep="\t", blocksize="0.1 GB")
elif file_format == "parquet":
    filename = os.path.join(DATA_DIR, "gene_info.parquet")
    genes = dd.read_parquet(os.path.join(DATA_DIR, "gene_info.parquet"), usecols=column_names, engine="pyarrow")
else:
    print("invalid file format")
    
print("Filename:", filename)
file_size = os.path.getsize(filename)
print(f"File Size: {file_size/1E9:.1f} GB")

genes = genes.rename(columns={"#tax_id": "tax_id"})

Filename: ../data/gene_info.tsv
File Size: 5.4 GB


### Process Data

In [8]:
genes = genes.query("type_of_gene == 'protein-coding'")
groups = genes.groupby("tax_id").size().reset_index()
groups.columns = ["tax_id", "count"]
groups = groups.sort_values("count", ascending=False)

Convert Dask to Pandas dataframe (this triggers the computation)

In [9]:
# groups = groups.compute(scheduler="processes") # use if Dask client is not instantiated
groups = groups.compute()

In [10]:
# terminate the Dask processes
client.close()

### Display Results

#### Number of human protein-coding genes (tax_id = 9606)

In [11]:
groups.query("tax_id == '9606'")

,tax_id,count
1462,9606,20598


#### Top 5 organisms with the most protein-coding genes

In [12]:
groups.head()

,tax_id,count
679,4565,104035
478,3708,90975
6979,90675,82686
7132,94328,68154
453,3635,67632


In [13]:
end = time.time()

In [14]:
print(f"Dask: {end - start:.1f} sec.")

Dask: 36.7 sec.
